# Code  Ensemble Learning with Spark

- Entire code for our 4 layers model 
- Case for temperature forecast on the Asport site
- Testbed for spark implementation

### Import of the useful packages 

In [1]:
import numpy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.io.json import json_normalize
from pandas.tseries import converter as pdtc
from sklearn.svm import SVR
import matplotlib.units as munits
from keras.layers import Dropout
from keras.regularizers import l2
from datetime import datetime
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from sklearn.linear_model import LinearRegression
from scipy import stats
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from functools import reduce
from datetime import datetime
from keras.optimizers import SGD
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas import read_csv
from pandas.plotting import scatter_matrix
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout, Lambda, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import chart_studio.plotly as py
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import keras
import json
import os
import sklearn
import numpy
from datetime import datetime
from dateutil.parser import parse
import findspark
findspark.init()
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession
import pyspark
pd.set_option('display.max_rows', None)

Using TensorFlow backend.


In [2]:
#Create spark context

spark = SparkSession \
        .builder \
        .appName("ensemble_learning_sparktest") \
        .getOrCreate()

In [3]:

#enables pyArrow optimization for dataframe conversions FIX THIS

#spark.conf.set("spark.sql.execution.arrow.enabled", "true")

### Extract JSON files informations

In [4]:
def concatenate_json_to_df(directory_path, to_output, folder_output):
    list_json = []
    
    for file in os.listdir(directory_path):
        if file.split(".")[-1] == "json":
            
            with open(directory_path+file, 'r') as f:
            
                # If json file is not empty
                if os.stat(directory_path+file).st_size != 0:
                                
                        
                    # For Observation file type
                    if directory_path.split("/")[-2] == "Observations":
                        dictio = json.load(f)
                                            
                        if 'observation' in dictio:
                            observation = dictio["observation"]
                        
                            for entry in observation:
                                entry['date'] =  file.rstrip(".json")
                                entry['station'] = dictio['station']
                                entry['time'] = datetime.utcfromtimestamp(entry['time']).strftime('%d-%m-%Y %H:%M:%S')
                                list_json.append(entry)
                    
                    
                    # For Analyses file type
                    elif directory_path.split("/")[-2] == "Analyses":
                        dictio = json.load(f)
                        if 'source' in dictio and 'temperature' in dictio:
                            source = list(dictio['source'])
                            temperature = list(dictio['temperature'])
                            
                            dictio['date'] = file.rstrip(".json")
                            dictio.pop('source', None)
                            dictio.pop('temperature', None)
                            
                            entry = dict()
                            for i in range(0, len(source)):
                                dictio[source[i]] = temperature[i]
                                list_json.append(dictio)

                                
                    # For Previsions file type
                    elif "Previsions" in directory_path.split("/"):
                        dictio = json.load(f)
                        
                        latit_r = None
                        long_r = None
                        
                        if "longitude_r" in dictio:
                            long_r = dictio['longitude_r']

                        if "latitude_r" in dictio:
                            latit_r = dictio['latitude_r']
                            
                        if 'forecast' in dictio:
                            forecast = dictio['forecast'] 
                            
                            
                            for i in range(0, len(forecast)):
                                dictio = dict()
                                
                                if long_r != None and latit_r != None:
                                    dictio['latitude_r'] = latit_r
                                    dictio['longitude_r'] = long_r
                                
                                if "temperature" in forecast[i]:
                                    dictio["temperature"] = forecast[i]["temperature"]
                                    
                                if "time" in forecast[i]:
                                    dictio["time"] = datetime.utcfromtimestamp(forecast[i]["time"]).strftime('%d-%m-%Y %H:%M:%S')
                                    
                                if "wind" in forecast[i]:
                                    dictio["wind"] = forecast[i]["wind"]
                                
                                if "wind_dir" in forecast[i]:
                                    dictio["wind_dir"] = forecast[i]["wind_dir"]
                                
                                if "humidity" in forecast[i]:
                                    dictio["humidity"] = forecast[i]["humidity"]
                                
                                if "pressure" in forecast[i]:
                                            dictio["pressure"] = forecast[i]["pressure"]
                                        
                                if "cloud_cover" in forecast[i]:
                                            dictio["cloud_cover"] = forecast[i]["cloud_cover"]

    
                                
                                dictio["date"] = file.rstrip(".json")
                                
                                list_json.append(dictio)
                
                
                else:
                    dictio = dict()
                    dictio["date"] = file.rstrip(".json")
                    list_json.append(dictio)
                                    
    df = pd.DataFrame(list_json)
    df.drop_duplicates()
    df.to_csv(folder_output+"/"+to_output.replace("/", "-")+".csv", sep=";", index=False)

In [5]:
dirname = "technical_challenge_v9"
if not os.path.exists(dirname):
    os.mkdir(dirname)

In [6]:
#testing the formatting function for one folder
concatenate_json_to_df('technical_challenge/Asport CAPRED/Previsions/Api-Agro/', "Asport CAPRED/Previsions/Api-Agro", dirname)

In [8]:
path = "technical_challenge/"
for directories in os.listdir(path):
       
    for folder in os.listdir(path+directories):
           
        if folder == "Previsions":
            for folder2 in os.listdir(path+directories+"/"+folder):
                
                concatenate_json_to_df(path+directories+"/"+folder+"/"+folder2+"/", directories+"/"+folder+"/"+folder2, dirname)
                print(path+directories+"/"+folder+"/"+folder2)
        else: 
            print(path+directories+"/"+folder)
            concatenate_json_to_df(path+directories+"/"+folder+"/", directories+"/"+folder, dirname)
            

NotADirectoryError: [Errno 20] Not a directory: 'technical_challenge/.DS_Store'

### Launch and transformation of data

In [9]:
# Launch data of Api, Dark and weatherbit forcesaters for the station Asport
path = "technical_challenge_v9/Asport CAPRED-Previsions-"
path_previsions = ["Api-Agro.csv", "Dark Sky.csv", "Weatherbit.csv"]
path_observations = "technical_challenge_v9/Asport CAPRED-Observations.csv"

In [10]:
# Transform the observation and prevision data 
def get_forecast(path, path_previsions, path_observations):
    list_data = []
    for forecaster in path_previsions:
        dataframe = pd.read_csv(path + forecaster, sep=";")
        dataframe["time"] = pd.to_datetime(dataframe['time'], format='%d-%m-%Y %H:%M:%S')
        dataframe.sort_values(by=['time'])
        dataframe['time'] = dataframe['time'].apply(lambda x: x.replace(minute=0, second=0))
        dataframe = dataframe.groupby(['time']).apply(lambda x : x.iloc[:6].mean())
        dataframe.columns = [x + "_" + forecaster.split('.')[0] for x in dataframe.columns]
        list_data.append(dataframe)   
    
    observation_data = pd.read_csv(path_observations, sep=";", index_col=False).drop(columns=["station"]).dropna()
    observation_data["time"] = pd.to_datetime(observation_data['time'], format='%d-%m-%Y %H:%M:%S')
    observation_data.sort_values(by='time')
    observation_data['time'] = observation_data['time'].apply(lambda x: x.replace(minute=0, second=0))
    observation_data = observation_data.groupby(['time']).apply(lambda x : x.iloc[0])
    observation_data=observation_data.rename(index=str, columns={"humidity": "humidity_Y", "precipitation": "precipitation_Y", "temperature":"temperature_Y"})
    
    return list_data, observation_data

In [11]:
# Associate X to previsions and Y to observations 
list_data, observation_data = get_forecast(path, path_previsions, path_observations)
X = pd.concat(list_data, axis=1)
Y = observation_data

In [12]:
# Delete non useful variable in prevision data
X = X.loc[:, ~X.columns.str.startswith('l')]
X = X.loc[:, ~X.columns.str.startswith('c')]
X = X.loc[:, ~X.columns.str.startswith('p')]
X = X.loc[:, ~X.columns.str.startswith('wind_dir')]
X = X.loc[:, ~X.columns.str.startswith('prec')]
X = X.loc[:, ~X.columns.str.startswith('date')]
X = X.loc[:, ~X.columns.str.startswith('time')]

In [13]:
# Delete non useful variable in observation data
Y = Y.drop(['date','time'], axis=1)

In [14]:
# Import JSON API in order to collect most recent data
import json,urllib.request
data_A = urllib.request.urlopen("http://sd-59247.dedibox.fr/agriscope/obs/transform_obs.php?STATION=Asport").read()
output_A = json.loads(data_A)

In [15]:
# Transform JSON data in a dataframe A
A = json.loads(data_A)
A_data = pd.DataFrame(A)
A = A_data.T

In [16]:
# Delelte the index of the dataframe A
A.index.name = 'time'
A.reset_index(level=0, inplace=True)

In [17]:
# Convert date time in correct format 
A['time'] = pd.to_datetime(A['time'])
A['time'] = pd.to_datetime(A['time'],'%d-%m-%Y %H:%M:%S')
A['time'] = A['time'].apply(lambda x: x.replace(minute=0, second=0))

In [18]:
# Rename dataframe A columns
A.columns = ['time', 'humidity_Y', 'precipitation_Y', 'temperature_Y']

In [19]:
# Reset the index of the dataframe A
A = A.set_index('time') 

In [20]:
# Concat vertically Y and A (Y contains former observation and A most recent one)
frames = [Y, A]
first_concat = pd.concat(frames)

In [21]:
# Merge X and Y dataframe in one single dataframe "df_forcast_cap"
df_forcast_cap = pd.merge(X, first_concat, right_index=True, left_index=True)
df_forcast_cap = df_forcast_cap.drop(['precipitation_Y'], axis=1)

In [22]:
# Delete missing values for "df_forcast_cap"
df_forcast_cap = df_forcast_cap.dropna()

In [23]:
df_forcast_cap.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2131 entries, 2019-03-01 01:00:00 to 2019-08-23 18:00:00
Data columns (total 11 columns):
temperature_Api-Agro      2131 non-null float64
wind_Api-Agro             2131 non-null float64
humidity_Api-Agro         2131 non-null float64
humidity_Dark Sky         2131 non-null float64
temperature_Dark Sky      2131 non-null float64
wind_Dark Sky             2131 non-null float64
humidity_Weatherbit       2131 non-null float64
temperature_Weatherbit    2131 non-null float64
wind_Weatherbit           2131 non-null float64
humidity_Y                2131 non-null float64
temperature_Y             2131 non-null float64
dtypes: float64(11)
memory usage: 199.8+ KB


# Couche 1

### Random forest (RF)

In [30]:
X = df_forcast_cap.drop(['temperature_Y','humidity_Y'],axis=1)
y = df_forcast_cap['temperature_Y']

# creation of X train and test 
X_train = int(len(X) * 0.75)
X_train, X_test = X[0:X_train], X[X_train:len(X)]

# creation of y train and test 
y_train = int(len(y) * 0.75)
y_train, y_test = y[0:y_train], y[y_train:len(y)]

# Information about train and test and X variables
print('X: %d' % (len(X)))
print('Training X: %d' % (len(X_train)))
print('Testing X: %d' % (len(X_test)))

X: 2131
Training X: 1598
Testing X: 533


In [31]:
#convert pandas to dataframe

df_forecast_cap_SPARK = spark.createDataFrame(df_forcast_cap) #dataframe
train_dataset  = df_forecast_cap_SPARK.rdd()

print(df_forecast_cap_SPARK.toPandas())

      temperature_Api-Agro  wind_Api-Agro  humidity_Api-Agro  \
0                 9.857581       6.529342          64.971182   
1                 9.590286       7.193256          65.072065   
2                 9.771405       6.694767          62.447373   
3                 9.864105       8.528042          60.785817   
4                 9.651237       8.649829          62.393510   
5                 9.386034       8.479912          64.240130   
6                 9.621805       7.959605          64.717417   
7                10.888361       8.313673          60.373382   
8                12.697982       8.416563          54.578627   
9                14.137156      10.323657          48.780962   
10               15.752909      10.934787          42.881544   
11               17.132898      10.013908          38.651726   
12               17.853418      11.309892          36.693719   
13               18.221470      11.001425          36.330678   
14               18.039337      11.21190

In [27]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')

pprint(rf.get_params())

#Trains random forest. note that numTrees is low for testing purposes
model = RandomForest.trainRegressor(df_forecast_cap_SPARK, categoricalFeaturesInfo={},
                                        numTrees=3, featureSubsetStrategy="auto",
                                        impurity='variance', maxDepth=4, maxBins=32)

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


AssertionError: the data should be RDD of LabeledPoint

In [32]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    predictions = model.predict(testData.map(lambda x: x.features))
    labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
    testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
        float(testData.count())
    print('Model Performance')
    print('Test Mean Squared Error = ' + str(testMSE))
    print('Learned regression forest model:')
    print(model.toDebugString())
    
    return testMSE

base_model = RandomForestRegressor(n_estimators = 10)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

NameError: name 'testData' is not defined

In [ ]:
y_pred_RF = base_model.predict(X_test)

# Print the RMSE test
print(np.sqrt((y_pred_RF - y_test) ** 2).mean())

# Print MSE
MSE = mean_squared_error(y_test, y_pred_RF) 
print(MSE)

### Multilayer Perceptron (MLP)

In [ ]:
model = Sequential()
model.add(Dense((10), input_dim=9, activation='linear', kernel_initializer='normal', 
                kernel_regularizer=l2(0.03), bias_regularizer=l2(0.03)))
model.add(Dense(6, kernel_initializer='normal', activation='linear',
                kernel_regularizer=l2(0.03), bias_regularizer=l2(0.03)))
model.add(Dense(1, activation='linear'))

# compile model
model.compile(loss='mean_squared_error', optimizer='Adagrad')

# fit model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3000, batch_size=7, verbose=0)

# evaluate the model
train_mse = model.evaluate(X_train, y_train, verbose=0)
test_mse = model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_mse, test_mse))

# plot loss during training
plt.figure(figsize=(10,7))
pyplot.title('Mean Squared Error')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

print()

In [ ]:
# Print and save predicted values 
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)

# Reshape preds_train & preds_test (27, 1)  in an array (2D -> 1D)
predict_train_MLP = preds_train.flatten()
predict_test_MLP = preds_test.flatten()

In [ ]:
# RMSE & MSE 
# Print the RMSE train
print(np.sqrt((predict_train_MLP - y_train) ** 2).mean())
# Print the RMSE test
print(np.sqrt((predict_test_MLP - y_test) ** 2).mean())

# Print MSE
MSE_train = mean_squared_error(y_train, predict_train_MLP) 
print(MSE_train)
MSE_test = mean_squared_error(y_test, predict_test_MLP) 
print(MSE_test)

### Gradient Boosting Regressor (GBR)

In [ ]:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 
          'gamma':[i/10.0 for i in range(3,6)],
          'subsample':[i/10.0 for i in range(6,11)],
            'colsample_bytree':[i/10.0 for i in range(6,11)],
          'max_depth': [2,3,4]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(X_train, y_train)

y_pred_XGBR_train = grid.best_estimator_.predict(X_train)
y_pred_XGBR_test = grid.best_estimator_.predict(X_test)

In [ ]:
# RMSE & MSE 
# Print the RMSE test
print(np.sqrt((y_pred_XGBR_test - y_test) ** 2).mean())

# Print MSE
MSE_test = mean_squared_error(y_test, y_pred_XGBR_test) 
print(MSE_test)

# Couche 2

### RF

In [ ]:
# Concatenate results from RF, GBR, MLP, SVR, create X and y for Stacking layer
RF = pd.DataFrame(y_pred_RF)
MLP = pd.DataFrame(predict_test_MLP)
GBR = pd.DataFrame(y_pred_XGBR_test)

X_stack = pd.concat([RF, MLP, GBR], axis=1, join='inner')
X_stack.columns = ['RF','MLP','GBR']
y_stack = y_test

In [ ]:
# Repartition of data by respecting the series order
# creation of X train and test 
X_stack_train = int(len(X_stack) * 0.75)
X_stack_train, X_stack_test = X_stack[0:X_stack_train], X_stack[X_stack_train:len(X)]

# creation of y train and test 
y_stack_train = int(len(y_stack) * 0.75)
y_stack_train, y_stack_test = y_stack[0:y_stack_train], y_stack[y_stack_train:len(y)]

# Information about train and test and X variables
print('X: %d' % (len(X_stack)))
print('Training X: %d' % (len(X_stack_train)))
print('Testing X: %d' % (len(X_stack_test)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')

pprint(rf.get_params())
rf.fit(X_stack_train, y_stack_train)

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10)
base_model.fit(X_stack_train, y_stack_train)
base_accuracy = evaluate(base_model, X_stack_test, y_stack_test)

In [ ]:
y_stack_pred_RF = base_model.predict(X_stack_test)

# Print the RMSE test
print(np.sqrt((y_stack_pred_RF - y_stack_test) ** 2).mean())

# Print MSE
MSE = mean_squared_error(y_stack_test, y_stack_pred_RF) 
print(MSE)

### MLP

In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=3, activation='relu', kernel_initializer='normal',
                kernel_regularizer=l2(0.04), bias_regularizer=l2(0.04)))
model.add(Dense(6, kernel_initializer='normal', activation='linear',
                kernel_regularizer=l2(0.04), bias_regularizer=l2(0.04)))
model.add(Dense(1, activation='linear'))

# compile model
model.compile(loss='mean_squared_error', optimizer='Adagrad')

# fit model
history = model.fit(X_stack_train, y_stack_train, validation_data=(X_stack_test, y_stack_test), epochs=1000, batch_size=10, verbose=0)

# evaluate the model
train_mse = model.evaluate(X_stack_train, y_stack_train, verbose=0)
test_mse = model.evaluate(X_stack_test, y_stack_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_mse, test_mse))

# plot loss during training
plt.figure(figsize=(10,7))
pyplot.title('Mean Squared Error')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

print()

In [ ]:
# Print and save predicted values 
preds_train = model.predict(X_stack_train)
preds_test = model.predict(X_stack_test)

# Reshape preds_train & preds_test (27, 1)  in an array (2D ->1D)
predict_train_stack_MLP = preds_train.flatten()
predict_test_stack_MLP = preds_test.flatten()

In [ ]:
# RMSE & MSE 
# Print the RMSE train
print(np.sqrt((predict_train_stack_MLP - y_stack_train) ** 2).mean())
# Print the RMSE test
print(np.sqrt((predict_test_stack_MLP - y_stack_test) ** 2).mean())

# Print MSE
MSE_train = mean_squared_error(y_stack_train, predict_train_stack_MLP) 
print(MSE_train)
MSE_test = mean_squared_error(y_stack_test, predict_test_stack_MLP) 
print(MSE_test)

### GBR

In [ ]:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 
          'gamma':[i/10.0 for i in range(3,6)],
          'subsample':[i/10.0 for i in range(6,11)],
            'colsample_bytree':[i/10.0 for i in range(6,11)],
          'max_depth': [2,3,4]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(X_stack_train, y_stack_train)

y_pred_XGBR_train_stack = grid.best_estimator_.predict(X_stack_train)
y_pred_XGBR_test_stack = grid.best_estimator_.predict(X_stack_test)

In [ ]:
# RMSE & MSE 
# Print the RMSE test
print(np.sqrt((y_pred_XGBR_test_stack - y_stack_test) ** 2).mean())

# Print MSE
MSE_test = mean_squared_error(y_stack_test, y_pred_XGBR_test_stack) 
print(MSE_test)

# Couche 3

### RF

In [ ]:
# Concatenate results from RF, GBR, MLP, SVR, create X and y for Stacking layer
RF_stack = pd.DataFrame(y_stack_pred_RF)
MLP_stack = pd.DataFrame(predict_test_stack_MLP)
GBR_stack = pd.DataFrame(y_pred_XGBR_test_stack)

X_stack_final = pd.concat([RF_stack,MLP_stack,GBR_stack], axis=1, join='inner')
X_stack_final.columns = ['RF_stack','MLP_stack','GBR_stack']
y_stack_final = y_stack_test

In [ ]:
# Repartition of data by respecting the series order
# creation of X train and test 
X_stack_train_final = int(len(X_stack_final) * 0.75)
X_stack_train_final, X_stack_test_final = X_stack_final[0:X_stack_train_final], X_stack_final[X_stack_train_final:len(X_stack_final)]

# creation of y train and test 
y_stack_train_final = int(len(y_stack_final) * 0.75)
y_stack_train_final, y_stack_test_final = y_stack_final[0:y_stack_train_final], y_stack_final[y_stack_train_final:len(y_stack_final)]

# Information about train and test and X variables
print('X: %d' % (len(X_stack_final)))
print('Training X: %d' % (len(X_stack_train_final)))
print('Testing X: %d' % (len(X_stack_test_final)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')

pprint(rf.get_params())
rf.fit(X_stack_train_final, y_stack_train_final)

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10)
base_model.fit(X_stack_train_final, y_stack_train_final)
base_accuracy = evaluate(base_model, X_stack_test_final, y_stack_test_final)

In [ ]:
y_stack_pred_RF_final = base_model.predict(X_stack_test_final)

# Print the RMSE test
print(np.sqrt((y_stack_pred_RF_final - y_stack_test_final) ** 2).mean())

# Print MSE
MSE = mean_squared_error(y_stack_test_final, y_stack_pred_RF_final) 
print(MSE)

### MLP

In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=3, activation='linear', kernel_initializer='normal',
                kernel_regularizer=l2(0.04), bias_regularizer=l2(0.04)))
model.add(Dense(6, kernel_initializer='normal', activation='linear',
                kernel_regularizer=l2(0.04), bias_regularizer=l2(0.04)))
model.add(Dense(1, activation='linear'))

# compile model
model.compile(loss='mean_squared_error', optimizer='Adagrad')

# fit model
history = model.fit(X_stack_train_final, y_stack_train_final, validation_data=(X_stack_test_final, y_stack_test_final), epochs=(3000), batch_size=7, verbose=0)

# evaluate the model
train_mse = model.evaluate(X_stack_train_final, y_stack_train_final, verbose=0)
test_mse = model.evaluate(X_stack_test_final, y_stack_test_final, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_mse, test_mse))

# plot loss during training
plt.figure(figsize=(10,7))
pyplot.title('Mean Squared Error')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

print()

In [ ]:
# Print and save predicted values 
preds_train_final = model.predict(X_stack_train_final)
preds_test_final = model.predict(X_stack_test_final)

# Reshape preds_train & preds_test (27, 1)  in an array (2D ->1D)
predict_train_stack_MLP_final = preds_train_final.flatten()
predict_test_stack_MLP_final = preds_test_final.flatten()

In [ ]:
# RMSE & MSE 
# Print the RMSE train
print(np.sqrt((predict_train_stack_MLP_final - y_stack_train_final) ** 2).mean())
# Print the RMSE test
print(np.sqrt((predict_test_stack_MLP_final - y_stack_test_final) ** 2).mean())

# Print MSE
MSE_train = mean_squared_error(y_stack_train_final, predict_train_stack_MLP_final) 
print(MSE_train)
MSE_test = mean_squared_error(y_stack_test_final, predict_test_stack_MLP_final) 
print(MSE_test)

# Couche 4

### RF

In [ ]:
# Concatenate results from RF, GBR, MLP, SVR, create X and y for Stacking layer
RF_stack_final_2 = pd.DataFrame(y_stack_pred_RF_final)
MLP_stack_final_2 = pd.DataFrame(predict_test_stack_MLP_final)

X_stack_final_2 = pd.concat([RF_stack_final_2,MLP_stack_final_2], axis=1, join='inner')
X_stack_final_2.columns = ['RF_stack','MLP_stack']
y_stack_final_2 = y_stack_test_final

In [ ]:
# Repartition of data by respecting the series order
# creation of X train and test 
X_stack_final_2_train = int(len(X_stack_final_2) * 0.80)
X_stack_final_2_train, X_stack_final_2_test = X_stack_final_2[0:X_stack_final_2_train], X_stack_final_2[X_stack_final_2_train:len(X_stack_final_2)]

# creation of y train and test 
y_stack_final_2_train = int(len(y_stack_final_2) * 0.80)
y_stack_final_2_train, y_stack_final_2_test = y_stack_final_2[0:y_stack_final_2_train], y_stack_final_2[y_stack_final_2_train:len(y_stack_final_2)]

# Information about train and test and X variables
print('X: %d' % (len(X_stack_final_2)))
print('Training X: %d' % (len(X_stack_final_2_train)))
print('Testing X: %d' % (len(X_stack_final_2_test)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')

pprint(rf.get_params())
rf.fit(X_stack_final_2_train , y_stack_final_2_train)

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10)
base_model.fit(X_stack_final_2_train, y_stack_final_2_train)
base_accuracy = evaluate(base_model, X_stack_final_2_test, y_stack_final_2_test)

In [ ]:
y_stack_pred_RF_final_2_1 = base_model.predict(X_stack_final_2_test)

# Print the RMSE test
print(np.sqrt((y_stack_pred_RF_final_2_1  - y_stack_final_2_test) ** 2).mean())

# Print MSE
MSE = mean_squared_error(y_stack_final_2_test, y_stack_pred_RF_final_2_1) 
print(MSE)

In [ ]:
print(y_stack_pred_RF_final_2_1)

### MLP

In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=2, activation='linear', kernel_initializer='normal',
                kernel_regularizer=l2(0.03), bias_regularizer=l2(0.03)))

model.add(Dense(6, kernel_initializer='normal', activation='linear',
                kernel_regularizer=l2(0.03), bias_regularizer=l2(0.03)))

model.add(Dense(1, activation='linear'))

# compile model
model.compile(loss='mean_squared_error', optimizer='Adagrad')

# fit model
history = model.fit(X_stack_final_2_train, y_stack_final_2_train, validation_data=(X_stack_final_2_test, y_stack_final_2_test), epochs=1000, batch_size=7, verbose=0)

# evaluate the model
train_mse = model.evaluate(X_stack_final_2_train, y_stack_final_2_train, verbose=0)
test_mse = model.evaluate(X_stack_final_2_test, y_stack_final_2_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_mse, test_mse))

# plot loss during training
plt.figure(figsize=(10,7))
pyplot.title('Mean Squared Error')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

print()

In [ ]:
# Print and save predicted values 
preds_train_final = model.predict(X_stack_final_2_train)
preds_test_final = model.predict(X_stack_final_2_test)

# Reshape preds_train & preds_test (27, 1)  in an array (2D ->1D)
predict_train_stack_MLP_final = preds_train_final.flatten()
predict_test_stack_MLP_final = preds_test_final.flatten()

In [ ]:
# RMSE & MSE 
# Print the RMSE train
print(np.sqrt((predict_train_stack_MLP_final - y_stack_final_2_train) ** 2).mean())
# Print the RMSE test
print(np.sqrt((predict_test_stack_MLP_final - y_stack_final_2_test) ** 2).mean())

# Print MSE
MSE_train = mean_squared_error(y_stack_final_2_train, predict_train_stack_MLP_final) 
print(MSE_train)
MSE_test = mean_squared_error(y_stack_final_2_test, predict_test_stack_MLP_final) 
print(MSE_test)

## GBR

In [ ]:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 
          'gamma':[i/10.0 for i in range(3,6)],
          'subsample':[i/10.0 for i in range(6,11)],
            'colsample_bytree':[i/10.0 for i in range(6,11)],
          'max_depth': [2,3,4]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(X_stack_final_2_train, y_stack_final_2_train)

y_pred_XGBR_train = grid.best_estimator_.predict(X_stack_final_2_train)
y_pred_XGBR_test = grid.best_estimator_.predict(X_stack_final_2_test)

In [ ]:
# RMSE & MSE 
# Print the RMSE test
print(np.sqrt((y_pred_XGBR_test - y_stack_final_2_test) ** 2).mean())

# Print MSE
MSE_test = mean_squared_error(y_stack_final_2_test, y_pred_XGBR_test) 
print(MSE_test)

# Results and comparaisons 

In [ ]:
# Obtaine 7 last values prediction for each forcasters
prev = X.iloc[-7:]
prev = prev.loc[:, ~prev.columns.str.startswith('clou')]
prev = prev.loc[:, ~prev.columns.str.startswith('press')]
prev = prev.loc[:, ~prev.columns.str.startswith('wind')]
prev = prev.loc[:, ~prev.columns.str.startswith('hum')]
prev = prev.loc[:, ~prev.columns.str.startswith('pre')]

In [ ]:
# Transform observed values Serie in dataframe
observed = pd.DataFrame(data=y_stack_final_2)
observed = observed.iloc[-7:]
print(observed)

#Transform predicted value Serie in dataframe
predicted = pd.DataFrame(data=y_stack_pred_RF_final_2_1)
predicted.columns = ['predicted']
print(predicted)

In [ ]:
# Merge observed and prev
predictions = pd.merge(observed,prev, right_index=True, left_index=True)
predictions.reset_index(drop=True, inplace=True)
print(predictions)

In [ ]:
# Concatenate predictions and predicted
predictions_all = pd.concat([predictions,predicted], axis=1,ignore_index=True)
predictions_all.columns = ['Observed','Api-Agro','Dark_Sky','Weatherbit','predicted']
print(predictions_all)

In [ ]:
# Chart comparaison between predicted values and forcaster's values
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(12,8))
plt.plot(predictions_all)
plt.legend(predictions_all, fontsize=19)
plt.xlabel("Time slot", fontsize=17)
plt.ylabel("Temperature (C°)", fontsize=17)

In [ ]:
# Compute RMSE for each forcasters and predicted values
Observed = predictions_all['Observed']
Api = predictions_all['Api-Agro']
DarkSky = predictions_all['Dark_Sky']
Weatherbit = predictions_all['Weatherbit']
predicted = predictions_all['predicted']

RMSE_Dark = np.sqrt((DarkSky - Observed) ** 2).mean()
RMSE_Weather = np.sqrt((Weatherbit - Observed) ** 2).mean()
RMSE_Api = np.sqrt((Api - Observed) ** 2).mean()
RMSE_Predicted = np.sqrt((predicted - Observed) ** 2).mean()

RMSE_all = [RMSE_Dark,RMSE_Weather, RMSE_Api, RMSE_Predicted]
print(RMSE_all)